<a href="https://colab.research.google.com/github/ydg1021/basicRL/blob/main/example_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
# numpy는 수학적 연산을 위한 라이브러리입니다. 여기서는 배열과 행렬 연산에 사용됩니다.


In [ ]:
def policy_evaluation_sutton_converge(policy, transition_probabilities, rewards, gamma, theta=0.0001, print_iterations=None):
    # policy: 각 상태에서 취할 수 있는 행동의 확률
    # transition_probabilities: 상태와 행동에 따른 다음 상태로의 전이 확률
    # rewards: 각 행동에 대한 보상
    # gamma: 감가율(미래 보상의 현재 가치를 결정)
    # theta: 수렴 판단을 위한 임계값
    # print_iterations: 출력할 반복 횟수들의 목록

    num_states = policy.shape[0]  # 상태의 수
    num_actions = policy.shape[1]  # 행동의 수
    V = np.zeros(num_states)  # 상태 가치 초기화

    iteration = 0
    while True:
        iteration += 1  # 반복 카운터 증가
        delta = 0  # 가치 함수의 변화량
        V_updated = np.zeros(num_states)  # 업데이트된 가치 함수

        for s in range(num_states):
            if (s % grid_size[1], s // grid_size[1]) in terminal_states:
                continue  # 터미널 상태는 건너뜁니다.

            v = V[s]  # 현재 상태의 가치
            # 상태 가치 업데이트
            V_updated[s] = sum([policy[s, a] * sum([transition_probabilities[s, a, s_prime] *
                               (rewards[s, a, s_prime] + gamma * V[s_prime]) for s_prime in range(num_states)])
                               for a in range(num_actions)])
            delta = max(delta, abs(v - V_updated[s]))  # 최대 변화량 업데이트

        V = V_updated.copy()  # 상태 가치 업데이트
        # 터미널 상태의 가치를 0으로 설정
        for terminal_state in terminal_states:
            state_index = terminal_state[1] * grid_size[1] + terminal_state[0]
            V[state_index] = 0

        # 첫 번째 반복에서 비터미널 상태의 가치를 -1로 설정
        if iteration == 1:
            for x in range(grid_size[0]):
                for y in range(grid_size[1]):
                    if (x, y) not in terminal_states:
                        state = x * grid_size[1] + y
                        V[state] = -1

        # 지정된 반복 횟수에서 상태 가치 출력
        if print_iterations and iteration in print_iterations:
            print(f"Iteration {iteration}:")
            print(V.reshape(grid_size))
            print()  # 줄바꿈

        if delta < theta:  # 수렴 조건 검사
            break

    return V


In [ ]:
grid_size = (4, 4)  # 그리드 크기 설정 (4x4)
num_states = grid_size[0] * grid_size[1]  # 상태의 총 수 계산
terminal_states = [(0, 0), (3, 3)]  # 터미널 상태 설정
gamma = 1  # 감가율 설정

policy = np.full((num_states, 4), 0.25)  # 균등 무작위 정책 설정

# 전이 확률 및 보상 설정
transition_probabilities = np.zeros((num_states, 4, num_states))
rewards = np.zeros((num_states, 4, num_states))
# 각 상태와 행동에 대한 전이 확률 및 보상을 설정
for s in range(num_states):
    if (s % grid_size[1], s // grid_size[1]) in terminal_states:
        continue
    for a in range(4):
        next_s = s
        # 상, 하, 좌, 우 이동에 따른 다음 상태 결정
        if a == 0 and s // grid_size[1] > 0:  # Up
            next_s -= grid_size[1]
        elif a == 1 and s // grid_size[1] < grid_size[0] - 1:  # Down
            next_s += grid_size[1]
        elif a == 2 and s % grid_size[1] > 0:  # Left
            next_s -= 1
        elif a == 3 and s % grid_size[1] < grid_size[1] - 1:  # Right
            next_s += 1
        transition_probabilities[s, a, next_s] = 1
        rewards[s, a, next_s] = -1


In [ ]:
print_iterations = [1, 2, 3, 10, 20, 50]  # 출력할 반복 횟수 지정
estimated_values_converge = policy_evaluation_sutton_converge(policy, transition_probabilities, rewards, gamma, print_iterations=print_iterations)
print("Estimated values after convergence:")
print(estimated_values_converge.reshape(grid_size))  # 최종 상태 가치 출력
